### Mount vào dir chứa model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ModelAI/SpamFilter'

Mounted at /content/gdrive
/content/gdrive/MyDrive/ModelAI/SpamFilter


### Cài các thư viện cần thiết

In [ ]:
!pip install scikit-learn underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.2 MB/s eta 0:00:00


### Load dataset

In [ ]:
import os
DIR_ROOT = '/content/gdrive/MyDrive/ModelAI/SpamFilter'
DIR_DATASET = os.path.join(DIR_ROOT, 'dataset')

PATH_TRAIN = os.path.join(DIR_DATASET, 'train.csv')
PATH_DEV = os.path.join(DIR_DATASET, 'dev.csv')
PATH_TEST = os.path.join(DIR_DATASET, 'test.csv')

STOPWORDS_PATH = os.path.join(DIR_ROOT, 'vietnamese-stopwords.txt')


# Tạo dir model nếu chưa có
MODEL_DIR = os.path.join(DIR_ROOT, 'model')
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

### Load các từ trong tập train và tập test

In [ ]:
import pandas as pd

data_train = pd.read_csv(PATH_TRAIN)
data_test = pd.read_csv(PATH_TEST)
vnbooking_test = pd.read_csv(os.path.join(DIR_ROOT, 'vnbooking_dataset/train.csv'))
bnb_train_data = pd.read_csv(os.path.join(DIR_ROOT, 'bnb_dataset/translated_reviews.csv'))

# Tạo tập test từ vnbooking
x_test_bnb = vnbooking_test['Comments']
y_test_bnb = vnbooking_test['Label']
y_test_bnb.replace(2, 1, inplace=True)


# Chuẩn hóa data
y_test_replace = data_test['Label']
y_test_replace.replace(0, 1, inplace=True)
y_test_replace.replace(2, 1, inplace=True)
# Tạo tập test cuối cùng
x_test_train = pd.concat([data_test['Comment'], x_test_bnb], ignore_index=True)
y_test_train = pd.concat([y_test_replace, y_test_bnb], ignore_index=True)


# Tạo tập train (nhận hết các spam)
x_spam_train = data_train[data_train['Label'] == 1]['Comment']
y_spam_train = data_train[data_train['Label'] == 1]['Label']
# Tập train
bnb_x_train = bnb_train_data['Comments']
bnb_y_train = bnb_train_data['Label']

# Hoàn chỉnh tập train
complete_x_train = pd.concat([x_spam_train, bnb_x_train], ignore_index=True)
complete_y_train = pd.concat([y_spam_train, bnb_y_train], ignore_index=True)
complete_x_train = complete_x_train.fillna('')
complete_y_train = complete_y_train.fillna(1)

with open(STOPWORDS_PATH, 'r') as stopword_txt:
    list_stopwords = []
    for line in stopword_txt:
        #Đọc từng dòng trong txt và cho vào list
        word = line.strip('\n')
        list_stopwords.append(word)
    #Chuyển list stopword thành set
    list_stopwords = set(list_stopwords)

### Tiền xử lý dữ liệu với `underthesea`

In [ ]:
from underthesea import word_tokenize, pos_tag
import string, re


def filter_stop_word(tokenizers, list_stopwords):
    return [token for token in tokenizers if token not in list_stopwords]


# Lọc hết các emoji khỏi câu
def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


def pre_processing_data(text):
    if type(text) is not str: return text
    #Chuyển về chữ thường
    text = text.lower()
    #Loại bỏ dấu câu
    text = text.translate(str.maketrans("", "", string.punctuation))
    #Tách từ thành list token
    tokenizers = word_tokenize(text)
    #Lọc stopword
    tokenizers = filter_stop_word(tokenizers, list_stopwords)
    #Tạo câu hoàn chỉnh dựa trên stopword
    sentences = " ".join(tokenizers)
    #Loại bỏ hết emoji
    sentences = remove_emoji(sentences)
    #Tạm bỏ qua lemmetazion
    return sentences

### Tiến hành train

In [ ]:
#Import pipeline để chạy
from sklearn.pipeline import Pipeline
#Import các thư viện cần thiết
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #Vectorize hóa các từ trong văn bản
from sklearn.naive_bayes import MultinomialNB #Tiến hành dùng Naive Bayes để xử lý
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
#Import các hàm để đánh giá
from sklearn.metrics import classification_report, accuracy_score

complete_x_train = complete_x_train.apply(pre_processing_data)
x_test_bnb = x_test_bnb.apply(pre_processing_data)

#Các classifier sử dụng:
rf = RandomForestClassifier(n_estimators=1000, random_state=1000)
nb = MultinomialNB(class_prior=[0.5, 0.5])
#Kết hợp lại (Dùng thử Regression làm meta)
stacking = StackingClassifier(estimators=[('nb', nb), ('rf', rf)], final_estimator=LogisticRegression())

pline = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 10), max_features=85000)),  #Dùng Bag of Word
    ('classifier', stacking)
])


#Tiến hành train
pline.fit(complete_x_train, complete_y_train)


# Đánh giá trên tập test
y_pred_test = pline.predict(x_test_train)
print("Độ chính xác ở tập test:", accuracy_score(y_test_train, y_pred_test))
print(classification_report(y_test_train, y_pred_test))

Độ chính xác ở tập test: 0.9441249864381035
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      5231
           1       0.96      0.91      0.93      3986

    accuracy                           0.94      9217
   macro avg       0.95      0.94      0.94      9217
weighted avg       0.94      0.94      0.94      9217



###Đóng gói model lại

In [ ]:
import pickle
# Lưu model vào tệp .pkl
with open(os.path.join(MODEL_DIR, 'spamfilter_model.pkl'), 'wb') as f:
    pickle.dump(pline, f)

Note: Mở model lên

In [ ]:
import pickle
# Tải lại model từ tệp .pkl
with open(os.path.join(MODEL_DIR, 'spamfilter_model.pkl'), 'rb') as f:
    loaded_pipeline = pickle.load(f)

# Sử dụng model đã tải để dự đoán
y_predict_loaded = loaded_pipeline.predict_proba(["Thời gian lưu trú của chúng tôi với Hội tốt hơn 1000 lần so với những gì tôi tưởng tượng. Cảnh tượng và cảm giác của các ngọn núi xung quanh cùng với âm thanh vang vọng của gà trống gáy đã để lại ấn tượng lâu dài. Hội đãi chúng tôi những bữa ăn tuyệt vời ba lần một ngày cùng với trái cây mới hái từ cây mọc ở sân xung quanh. Sau khi nhận được những đề xuất tuyệt vời về các điểm tham quan trong khu vực, Hội đã cung cấp một dịch vụ vận chuyển tuyệt vời để đưa chúng tôi đến bất cứ nơi nào chúng tôi chọn. Bất cứ ai đang tìm kiếm một nơi thư giãn thanh bình, cùng với dịch vụ tuyệt vời, Yen Binh Homestay nằm ở đầu danh sách nơi ở của tôi."])
print(y_predict_loaded)

[[0.98947216 0.01052784]]
